https://devblogs.nvidia.com/deep-learning-self-driving-cars/

In [1]:
%matplotlib inline
%load_ext autoreload
%autoreload 2

In [11]:
import numpy as np
import pandas as pd
import tensorflow as tf
import os
import pickle
from sklearn.utils import shuffle
from sklearn.model_selection import train_test_split
import cv2

tf.python.control_flow_ops = tf

from keras.models import Sequential
from keras.layers import Flatten, Dense, Lambda

In [36]:
import bclone

In [3]:
DATA_DIR = '../bclone-data-standard'

In [5]:
log_df = bclone.load_log(DATA_DIR)
n_samples = len(log_df) * 3

print('Number of samples (including all three cameras): ', n_samples)

Number of samples (including all three cameras):  24108


In [18]:
log_df_train, log_df_valid = train_test_split(log_df, test_size=0.2)

log_df_train.shape, log_df_valid.shape

((6428, 7), (1608, 7))

In [31]:
gen = bclone.data_generator(DATA_DIR, log_df_train, batch_size=4, controls=['steering'])
a, b = next(gen)
a.shape, b.shape

((12, 160, 320, 3), (12,))

In [7]:
model = Sequential()
model.add(Lambda(lambda x: x / 255. - 0.5, input_shape=(160, 320, 3)))
model.add(Flatten())
model.add(Dense(1))
model.compile(loss='mse', optimizer='adam')

[How to Use Generators](https://classroom.udacity.com/nanodegrees/nd013/parts/fbf77062-5703-404e-b60c-95b78b2f3f9e/modules/6df7ae49-c61c-4bb2-a23e-6527e69209ec/lessons/46a70500-493e-4057-a78e-b3075933709d/concepts/b602658e-8a68-44e5-9f0b-dfa746a0cc1a)

In [45]:
train_gen = bclone.data_generator(DATA_DIR, log_df_train, batch_size=30, controls=['steering'])
valid_gen = bclone.data_generator(DATA_DIR, log_df_valid, batch_size=30, controls=['steering'])

In [38]:
history = bclone.fit_gen(model, train_gen, valid_gen, log_df_train, log_df_valid, n_epochs=2)

Epoch 1/2
19284/19284 [==============================] - 109s - loss: 2.7220 - val_loss: 2.8817
Epoch 2/2
19284/19284 [==============================] - 97s - loss: 5.6379 - val_loss: 5.9920


In [40]:
history.history

{'loss': [2.7220146269783814, 5.6379027895550591],
 'val_loss': [2.8816726367865035, 5.9920181213919799]}

In [46]:
bclone.fit_gen(model, train_gen, valid_gen, log_df_train, log_df_valid, n_epochs=2).history

Epoch 1/2
19284/19284 [==============================] - 97s - loss: 3.2417 - val_loss: 2.6504
Epoch 2/2
19284/19284 [==============================] - 100s - loss: 1.3617 - val_loss: 1.5931


{'loss': [3.2416583222117419, 1.3616580550130584],
 'val_loss': [2.6504420371197943, 1.5931009281037458]}